In [18]:
import os
from llama_parse import LlamaParse
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain_cohere import CohereRerank


DATA_DIR = "data/product_demo.pdf"
# Initialize Embeddings
embedding = FastEmbedEmbeddings(model_name="BAAI/bge-small-en-v1.5")

def load_data():
    loader = PDFPlumberLoader(DATA_DIR)
    documents = loader.load()

    # Split loaded documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=50)
    docs = text_splitter.split_documents(documents=documents)

    # initialize the bm25 retriever
    bm25_retriever = BM25Retriever.from_documents(docs)
    bm25_retriever.k = 3
    
    #initialize the chroma retriever
    faiss_vectorstore = FAISS.from_documents(docs, embedding)
    faiss_retrieval = faiss_vectorstore.as_retriever(search_kwargs={"k": 3})

    # fusion FAISS and BM25
    ensemble_retriever = EnsembleRetriever(retrievers=[faiss_retrieval, bm25_retriever],
                                           weights=[0.5, 0.5])
    
    # rerank with cohere
    compressor = CohereRerank(cohere_api_key="SKDNVMyQRo7ZlAyziWOIpPYYrG9E7XzDMVPmQtH5")
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor, 
        base_retriever=ensemble_retriever
    )

    return compression_retriever, embedding



Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [26]:
from groq import Groq 
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_community.vectorstores import FAISS
from langchain_community.docstore import InMemoryDocstore
import faiss
from langchain.memory import ConversationBufferMemory
from langchain.memory import VectorStoreRetrieverMemory
from load_data import load_data
import gradio as gr

memory = ConversationBufferMemory(memory_key="history", input_key="question", return_messages=True)
# Initialize memory
memory.load_memory_variables({})

def set_custom_prompt():

    review_template_str = """
        Bạn là chuyên gia tư vấn khách hàng và rất am hiểu về các sản phẩm điện tử, gia dụng.... 
        Dựa vào thông tin được cung cấp và câu hỏi từ người dùng, hãy đưa ra câu trả lời đầy đủ và chính xác nhất.
        Nếu các thông tin trong câu hỏi không có trong phần được cung cấp trước đó, hãy nói "Xin lỗi, sản phẩm này chưa có trong cửa hàng của chúng tôi." và có thể gợi ý cho họ 1 số sản phẩm khác có trong cửa hàng.
        {context}

        Ngoài thông tin cho trước bạn có thể dựa vào các thông tin trong quá khứ cuộc trò chuyệnn để trả lời:
        {history}

        Lưu ý: 
        1. bạn chỉ được sử dụng tiếng việt để trả lời.
        2. Nếu câu hỏi không liên quan tới sản phẩm, chính sách, giá cả ... thì hãy tự trả lời dựa vào tri thức của bạn.

        """


    review_system_prompt = SystemMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=["context", "history"],
            template=review_template_str,
        )
    )

    review_human_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=["question"],
            template="{question}",
        )
    )
    messages = [review_system_prompt, review_human_prompt]

    prompt_template = ChatPromptTemplate(
        input_variables=["context", "question", "history"],
        messages=messages)
    return prompt_template


# Instantiate the Retrieval Question Answering Chain
def create_Chain_QA():
    llm = ChatGroq(
        model='Llama3-70b-8192',
        api_key="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"
    )

    prompt = set_custom_prompt()
    compression_retriever, embed_model = load_data()

    # embedding_size = 384 # dimension of the embedding model
    # index =faiss.IndexFlatL2(embedding_size)
    # vector_store = FAISS(embed_model.embed_query, 
    #                     index,
    #                     InMemoryDocstore({}), {})

    # # Create VectorStoreRetrieverMemory
    # retriever = vector_store.as_retriever(search_kwargs={"k": 2})
    # memory = VectorStoreRetrieverMemory(retriever=retriever, memory_key="history", input_key="question", return_docs=True)

    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type='stuff',
        retriever=compression_retriever,
        verbose=True,
        chain_type_kwargs={
            "verbose": True,
            "prompt": prompt,
            "memory": memory
        }
    )
    return qa

chain = create_Chain_QA()

# Function to handle chat messages
def chat(input_text, history=[]):
    # Load the current conversation history
    # memory.load_memory_variables({})
    inputs = {"question": input_text}
    response = chain.invoke(input_text)
    # Save the conversation in memory
    memory.save_context(inputs, {"output": response['result']})
    
    # Append the new interaction to the history
    history.append((input_text, response['result']))
    return response['result'], history


# # Create Gradio interface
# with gr.Blocks() as iface:
#     chatbot = gr.Chatbot()
#     msg = gr.Textbox(label="Gõ tin nhắn")
#     clear = gr.Button("Clear")

#     def user_interaction(user_message, history):
#         bot_response, updated_history = chat(user_message, history)
#         return updated_history, updated_history

#     msg.submit(user_interaction, [msg, chatbot], [chatbot, chatbot])
#     clear.click(lambda: None, None, chatbot, queue=False)

# # Launch the interface
# iface.launch()


response = chain.invoke("Tôi cần mua điều hòa")
print(response['result'])



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
        Bạn là chuyên gia tư vấn khách hàng và rất am hiểu về các sản phẩm điện tử, gia dụng.... 
        Dựa vào thông tin được cung cấp và câu hỏi từ người dùng, hãy đưa ra câu trả lời đầy đủ và chính xác nhất.
        Nếu các thông tin trong câu hỏi không có trong phần được cung cấp trước đó, hãy nói "Xin lỗi, sản phẩm này chưa có trong cửa hàng của chúng tôi." và có thể gợi ý cho họ 1 số sản phẩm khác có trong cửa hàng.
        - 509380: Điều hòa treo tường Aqua AQA-RV13QC
- 509384: Điều hòa treo tường Aqua AQA-KCR9PA
- 509383: Điều hòa treo tường Aqua AQA-KCR12NQ-S
- 509168: Điều hòa MDV 9000 BTU M&EMĐH000325/M&EMĐH000326
- 509170: Điều hòa MDV 18000 BTU M&EMĐH000329/M&EMĐH000330
- 509379: Điều hòa treo tường Aqua AQA-RV9QC
- 509381: Điều hòa treo tường Aqua AQA-RV18QA
9. Đèn Năng Lượng Mặt Trời
Số lượng sản phẩm là: 40
- 509175: 

In [27]:
response = chain.invoke("Tôi vừa hỏi bạn điều gì ?")
print(response['result'])



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
        Bạn là chuyên gia tư vấn khách hàng và rất am hiểu về các sản phẩm điện tử, gia dụng.... 
        Dựa vào thông tin được cung cấp và câu hỏi từ người dùng, hãy đưa ra câu trả lời đầy đủ và chính xác nhất.
        Nếu các thông tin trong câu hỏi không có trong phần được cung cấp trước đó, hãy nói "Xin lỗi, sản phẩm này chưa có trong cửa hàng của chúng tôi." và có thể gợi ý cho họ 1 số sản phẩm khác có trong cửa hàng.
        - 509231: Nồi chiên không dầu Philips công suất 2000W, sử dụng chiên 2 kg, kết nối điều
khiển thông qua ứng dụng Nutri U, chức năng Quick clean giúp vệ sinh dễ dàng, có thể sử dụng
trong máy rửa chén. HD9880/90
- 509228: Nồi chiên không dầu Philips công suất 2000 W, với công nghệ Rapid Air lành
mạnh với sức khỏe, làm đều nhiệt, sử dụng chiên cho 1,2 kg thực phẩm, có kết nối và điều khiển
qua Wifi và app Nutr

In [28]:
memory.buffer

[HumanMessage(content='Tôi cần mua điều hòa'),
 AIMessage(content='Điều hòa đang là sản phẩm rất hot hiện nay!\n\nTrong cửa hàng của chúng tôi, có các loại điều hòa treo tường Aqua sau:\n\n* Điều hòa treo tường Aqua AQA-RV13QC (mã sản phẩm: 509380)\n* Điều hòa treo tường Aqua AQA-KCR9PA (mã sản phẩm: 509384)\n* Điều hòa treo tường Aqua AQA-KCR12NQ-S (mã sản phẩm: 509383)\n* Điều hòa treo tường Aqua AQA-RV9QC (mã sản phẩm: 509379)\n* Điều hòa treo tường Aqua AQA-RV18QA (mã sản phẩm: 509381)\n\nNgoài ra, chúng tôi còn có điều hòa MDV với công suất 9000 BTU (mã sản phẩm: 509168) và 18000 BTU (mã sản phẩm: 509170).\n\nBạn đang tìm kiếm điều hòa với công suất bao nhiêu? Hay bạn có yêu cầu nào khác về điều hòa?'),
 HumanMessage(content='Tôi vừa hỏi bạn điều gì ?'),
 AIMessage(content='Bạn vừa hỏi tôi "Tôi vừa hỏi bạn điều gì ?"')]

In [135]:
from lib import *
from load_data import load_data

dotenv.load_dotenv()


class ChatBot:
    def __init__(self):
        self.prompt = self.set_custom_prompt()
        self.compression_retriever, self.embed_model = load_data()
        self.LLM = ChatGroq(
            model="llama3-70b-8192",
            api_key=os.getenv("GROQ_API_KEY"),
            temperature=0.0,
        )
        # self.memory = self.create_memory()
        # Initialize memory
        # self.memory.load_memory_variables({})
        
        self.chain = self.create_Chain_QA()

    # def create_memory(self):
    #     embedding_size = 384 # dimension of the embedding model
    #     index =faiss.IndexFlatL2(embedding_size)
    #     vector_store = FAISS(self.embed_model.embed_query, 
    #                         index,
    #                         InMemoryDocstore({}), {})

    #     # Create VectorStoreRetrieverMemory
    #     retriever = vector_store.as_retriever(search_kwargs={"k": 2})
    #     memory = VectorStoreRetrieverMemory(retriever=retriever, memory_key="history", input_key="question", return_docs=True)

    #     return memory


    def set_custom_prompt(self):
        review_template_str = """
        Bạn là chuyên gia tư vấn khách hàng và rất am hiểu về các sản phẩm điện tử, gia dụng.... 
        Dựa vào thông tin được cung cấp và câu hỏi từ người dùng, hãy đưa ra câu trả lời đầy đủ và chính xác nhất.
        Nếu các thông tin trong câu hỏi không có trong phần được cung cấp trước đó, hãy nói "Xin lỗi, sản phẩm này chưa có trong cửa hàng của chúng tôi." và có thể gợi ý cho họ 1 số sản phẩm khác có trong cửa hàng.
        {context}
        Lưu ý: 
        1. bạn chỉ được sử dụng tiếng việt để trả lời.
        2. Nếu câu hỏi không liên quan tới sản phẩm, chính sách, giá cả ... thì hãy tự trả lời dựa vào tri thức của bạn.

        """

        review_system_prompt = SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=["context"],
                template=review_template_str,
            )
        )

        review_human_prompt = HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=["question"],
                template="{question}",
            )
        )
        messages = [review_system_prompt, review_human_prompt]

        prompt_template = ChatPromptTemplate(
            input_variables=["context", "question"],
            messages=messages)
        
        return prompt_template


    # Instantiate the Retrieval Question Answering Chain
    def create_Chain_QA(self):

        qa = RetrievalQA.from_chain_type(
        llm=self.LLM,
        chain_type='stuff',
        retriever=self.compression_retriever,
        verbose=True,
        chain_type_kwargs={
            "verbose": True,
            "prompt": self.prompt,
            "memory": ConversationBufferMemory(memory_key="history", input_key="question")
            }
        )

        return qa

  # Function to handle chat messages
    def chat(self, input_text, history=[]):
        self.memory.load_memory_variables({})
        inputs = {"input": input_text}

        response = self.chain.invoke(input_text)
        # Save the conversation in memory
        self.memory.save_context(inputs, {"output": response['result']})
        
        # Append the new interaction to the history
        history.append((input_text, response['result']))
        return response['result'], history
    
    # Create Gradio interface
    def run_app(self):
        with gr.Blocks() as iface:
            chatbot = gr.Chatbot()
            msg = gr.Textbox(label="Gõ tin nhắn")
            clear = gr.Button("Clear")

            def user_interaction(user_message, history):
                bot_response, updated_history = self.chat(user_message, history)
                return updated_history, updated_history

            msg.submit(user_interaction, [msg, chatbot], [chatbot, chatbot])
            clear.click(lambda: None, None, chatbot, queue=False)

        # Launch the interface
        iface.launch()

if __name__ == "__main__":
    bot = ChatBot()
    # bot.run_app()

    response = bot.chain.invoke("bên bạn có sản phẩm tủ lạnh toshiba không ?")
    print(response['result'])





> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
        Bạn là chuyên gia tư vấn khách hàng và rất am hiểu về các sản phẩm điện tử, gia dụng.... 
        Dựa vào thông tin được cung cấp và câu hỏi từ người dùng, hãy đưa ra câu trả lời đầy đủ và chính xác nhất.
        Nếu các thông tin trong câu hỏi không có trong phần được cung cấp trước đó, hãy nói "Xin lỗi, sản phẩm này chưa có trong cửa hàng của chúng tôi." và có thể gợi ý cho họ 1 số sản phẩm khác có trong cửa hàng.
        I. Các loại sản phẩm
1. Bếp từ
Số lượng sản phẩm là: 10
- 606015: Bộ nồi Inox sơn đỏ (3 cái/Bộ)-AIO Smart
- 509333: Bếp Từ Bluestone Icb-6619
- 509331: Bếp Từ Bluestone Icb-6609
- 509330: Bếp Từ Đôi Bluestone Icb-6833
- 509328: Bếp Từ Đôi Bluestone Icb-6818
- 509336: Bếp Hỗn Hợp Quang Từ Bluestone Icb- 6917
- 509337: Bếp Từ Đôi Bluestone Icb-6948
- 509335: Bếp Hỗn Hợp Quang Từ Bluestone Icb-6911
- 509334: Bế

In [136]:
response = bot.chain.invoke("tôi vừa nói với bạn điều gì vậy ?")
print(response['result'])



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
        Bạn là chuyên gia tư vấn khách hàng và rất am hiểu về các sản phẩm điện tử, gia dụng.... 
        Dựa vào thông tin được cung cấp và câu hỏi từ người dùng, hãy đưa ra câu trả lời đầy đủ và chính xác nhất.
        Nếu các thông tin trong câu hỏi không có trong phần được cung cấp trước đó, hãy nói "Xin lỗi, sản phẩm này chưa có trong cửa hàng của chúng tôi." và có thể gợi ý cho họ 1 số sản phẩm khác có trong cửa hàng.
        - 509314: Nồi Cơm Điện Bluestone Rcb-5512
- 509316: Nồi Cơm Điện Bluestone Rcb-5538
- 509318: Nồi Cơm Điện Tử
- 509224: Nồi cơm điện Philips dung tích 1,8 lít công suất 790-940 W với 18 chương trình
dê dàng thiết lập giúp bạn đổi mới công thức nấu ăn hàng ngày vô cùng đa dạng. HD4518/62
- 509173: Nồi cơm điện KALITE KL-619, dung tích 1,8 lít
- 509174: Nồi cơm điện tử KALITE KL-620, dung tích 1,8 lít
- 509172

In [126]:
bot.memory.buffer

[]